In [1]:
import os           #파일접근
import glob         #파일 나열을위한
import numpy as np              # 다차원 배열, 수학함수
import matplotlib.pyplot as plt   #데이터 시각화 = 그래프 패키지
import pandas as pd     #데이터를 저장하고 처리하는 패키지
import json         # json
import base64 
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.models import load_model
from PIL import Image  # 이미지 클래스 사용

# 숫자 -> One-hot Vector 를 위한 라이브러리
from keras.utils import np_utils


### 데이터 load

In [2]:
labels = [ '트래디셔널', '매니시', '페미닌', '에스닉','컴템포러리',
          '내추럴', '젠더플루이드', '스포티', '서브컬쳐','캐주얼']

In [3]:
# labels = [ '레트로', '로맨틱', '리조트', '매니시','모던', '밀리터리',
#          '섹시', '소피스트케이티드', '스트리트', '스포티', '아방가르드',
#          '오리엔탈', '웨스턴', '젠더리스', '컨트리','클래식', '키치',
#          '톰보이', '펑크', '페미닌', '프레피', '히피', '힙합']
         

#####  전처리

In [4]:
# imageTest = Image.open('./K-Fashion/Training/원천데이터-기준10/내추럴/1155.jpg')
# baseheight = 500
# imageTest.size
# hpercent = (baseheight/float(imgor.size[1]))
#         wsize = int((float(imgor.size[0])*float(hpercent))) 
#         imgrs = imgor.resize((wsize, baseheight), Image.ANTIALIAS)




In [5]:
# 이미지 데이터 준비
x_train=[]
x_test=[]
y_train=[]
y_test=[]
cnt=0
path1='./K-Fashion/Training/원천데이터-기준10/'
for style in os.listdir(path1):
    path2=path1+style
    cnt=0
    print(style)
    styleN = labels.index(style)
    print(styleN)
    for img in os.listdir(path2):
        cnt=cnt+1
        path3=path2+'/'+img
        image=Image.open(path3)
        image = image.resize((250,250), resample=0) # 사이즈 조정 
        image_array = np.array(image) 
        if cnt <= 3000 :
            x_train.append(image_array)
            y_train.append(styleN)
        elif 3000 < cnt and cnt <= 3500:
            x_test.append(image_array)
            y_test.append(styleN)


내추럴
5
매니시
1
서브컬쳐
8
스포티
7
에스닉
3
젠더플루이드
6
캐주얼
9
컴템포러리
4
트래디셔널
0
페미닌
2


In [6]:
# # 정답
# y_train=[]
# y_test=[]
# cnt=0
# path1='./K-Fashion/Training/라벨링데이터1/'
# for style in os.listdir(path1):
#     path2=path1+style
#     cnt=0
#     print(style)
#     for json in os.listdir(path2):
#         cnt=cnt+1
#         path3=path2+'/'+json
        
#         if cnt < 2 :
#             print(json)
# #             y_train.append(image_array)
# #         elif 500 <= cnt and cnt < 1000:
# #             y_test.append(image_array)



In [7]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
y_train.shape

(26000,)

In [8]:
# Loss의 스케일 조정을 위해 0 ~ 255 -> 0 ~ 1 범위로 만들어줍니다
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# One-Hot Encoding (숫자 -> One-Hot Vector) 
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
# y의 최대값이 나오게된다
# [손흔드는 라이언] [오후 4:08] 아 현재 y의 값(라벨)이 3이라서
# [손흔드는 라이언] [오후 4:09] 0,1,2,3 이렇게 4개로 분류한다고
# [손흔드는 라이언] [오후 4:09] 되는걸까요?

In [9]:
x_train.shape, y_train.shape

((26000, 250, 250, 3), (26000, 10))

####  d

### 모델

In [10]:
width = 250
height = 250
channel = 3

# 모델 설정
# 순차적 모델을 만들기 위한 Sequential() 함수
model = models.Sequential()
# filters : Kernel 그룹의 갯수 / kernel_size : Kernel 크기
model.add(layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', 
                 input_shape=(width, height, 3)))
model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(layers.Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
# N차원 배열을 1차원으로 펴줍니다
model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# 출력층 MLP  # y 종류수 
model.add(layers.Dense(10,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 250, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 250, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 125, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                    

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='sgd', 
              metrics=['accuracy'])

In [12]:
# 100개 에폭 10, 배치 16  5시 45분 

In [13]:
hist = model.fit(x_train, y_train,
                 epochs=10,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                verbose=1,
                shuffle=True)

Epoch 1/10
1625/1625 [==============================] - 2025s 1s/step - loss: 2.1792 - accuracy: 0.1989 - val_loss: 1.9131 - val_accuracy: 0.3407
Epoch 2/10
1625/1625 [==============================] - 2218s 1s/step - loss: 1.9643 - accuracy: 0.3040 - val_loss: 1.6696 - val_accuracy: 0.4060
Epoch 3/10
1625/1625 [==============================] - 2123s 1s/step - loss: 1.7737 - accuracy: 0.3788 - val_loss: 1.8295 - val_accuracy: 0.3760
Epoch 4/10
1625/1625 [==============================] - 1994s 1s/step - loss: 1.6295 - accuracy: 0.4326 - val_loss: 1.6784 - val_accuracy: 0.4287
Epoch 5/10
1625/1625 [==============================] - 1849s 1s/step - loss: 1.4761 - accuracy: 0.4910 - val_loss: 1.6962 - val_accuracy: 0.4320
Epoch 6/10
1625/1625 [==============================] - 2020s 1s/step - loss: 1.3045 - accuracy: 0.5512 - val_loss: 1.7936 - val_accuracy: 0.4140
Epoch 7/10
1625/1625 [==============================] - 2044s 1s/step - loss: 1.1105 - accuracy: 0.6200 - val_loss: 1.9209 -

In [14]:
loss, acc = model.evaluate(x_test, y_test, verbose=1)

47/47 [==============================] - 29s 612ms/step - loss: 3.0489 - accuracy: 0.3227
